## Atribuir partidos às emendas

Este notebook carrega `emendas_por_favorecido.csv`, os partidos de deputados e os partidos de senadores a partir de dados abertos para associar o partido de cada autor de emenda sem alterar o arquivo original.

In [1]:
import pandas as pd
import json
from unidecode import unidecodefrom src.senado_party_service import fetch_senator_parties


In [2]:
# Baixa e prepara dados de partidos dos senadores
senadores = fetch_senator_parties()
senadores_df = pd.DataFrame(list(senadores.items()),
                            columns=['norm_name', 'siglaPartido'])

In [3]:
# Carrega partidos dos deputados a partir do JSON
with open('data/deputados_dados_partidos.json') as f:
    deputados = json.load(f)['dados']
deputados_df = pd.DataFrame(deputados)[['nome', 'siglaPartido']]
deputados_df['norm_name'] = deputados_df['nome'].apply(lambda x: unidecode(x).upper())

In [4]:
# Lê emendas e realiza merge sem alterar o CSV original
emendas = pd.read_csv('data/emendas_por_favorecido.csv')
emendas['norm_name'] = emendas['Nome do Autor da Emenda'].apply(lambda x: unidecode(x).upper())
partidos_df = pd.concat([deputados_df[['norm_name', 'siglaPartido']], senadores_df])
resultado = emendas.merge(partidos_df, on='norm_name', how='left')
resultado.drop(columns='norm_name', inplace=True)

In [5]:
# Salva nova versão sem alterar o arquivo original
resultado.to_csv('data/emendas_por_favorecido_partidos.csv', index=False)
resultado.head()